<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Exploratory-Analysis" data-toc-modified-id="Exploratory-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exploratory Analysis</a></span></li><li><span><a href="#Conduct-a-Statistical-Test" data-toc-modified-id="Conduct-a-Statistical-Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conduct a Statistical Test</a></span></li><li><span><a href="#Verifying-Results" data-toc-modified-id="Verifying-Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Verifying Results</a></span></li><li><span><a href="#$n\bullet-p-(1-p)$" data-toc-modified-id="$n\bullet-p-(1-p)$-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>$n\bullet p (1-p)$</a></span><ul class="toc-item"><li><span><a href="#Step-1:" data-toc-modified-id="Step-1:-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Step 1:</a></span></li><li><span><a href="#Step-2:" data-toc-modified-id="Step-2:-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Step 2:</a></span></li><li><span><a href="#Step-3:" data-toc-modified-id="Step-3:-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Step 3:</a></span></li><li><span><a href="#Analysis:" data-toc-modified-id="Analysis:-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Analysis:</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

In [2]:
#Your code here
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Preview the data
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
# Basic info about the dataset
print(df.info())
print(df['action'].value_counts())
print(df['group'].value_counts())

# 1. How many viewers also clicked?
# Pivot to get actions per ID
actions_pivot = df.pivot_table(index='id', columns='action', values='timestamp', aggfunc='count', fill_value=0)
viewers = actions_pivot[actions_pivot['view'] > 0].index
clickers = actions_pivot[actions_pivot['click'] > 0].index
viewers_who_clicked = len(set(viewers) & set(clickers))
print(f"Number of viewers who also clicked: {viewers_who_clicked}")

# 2. Check for anomalies (clicks without views)
clicks_no_view = actions_pivot[(actions_pivot['click'] > 0) & (actions_pivot['view'] == 0)]
print(f"Number of clicks without views: {len(clicks_no_view)}")
if len(clicks_no_view) > 0:
    print("Anomaly detected: Some IDs have clicks without views.")
    # Handle anomaly: Remove these IDs for analysis
    df = df[~df['id'].isin(clicks_no_view.index)]

# 3. Check for overlap between control and experiment groups
group_counts = df.groupby('id')['group'].nunique()
overlap_ids = group_counts[group_counts > 1].index
print(f"Number of IDs in both control and experiment groups: {len(overlap_ids)}")
if len(overlap_ids) > 0:
    print("Overlap detected: Some IDs appear in both groups.")
    # Handle overlap: Remove these IDs for clean analysis
    df = df[~df['id'].isin(overlap_ids)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
view     6328
click    1860
Name: action, dtype: int64
control       4264
experiment    3924
Name: group, dtype: int64
Number of viewers who also clicked: 1860
Number of clicks without views: 0
Number of IDs in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
#Your code here
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Create a binary variable: 1 for click, 0 for no click
# For each ID, check if they clicked
id_actions = df.pivot_table(index=['id', 'group'], columns='action', values='timestamp', aggfunc='count', fill_value=0).reset_index()
id_actions['clicked'] = id_actions['click'].apply(lambda x: 1 if x > 0 else 0)

# Split into control and experiment groups
control = id_actions[id_actions['group'] == 'control']
experiment = id_actions[id_actions['group'] == 'experiment']

# Calculate click-through rates
n_control = len(control)
n_experiment = len(experiment)
clicks_control = control['clicked'].sum()
clicks_experiment = experiment['clicked'].sum()
ctr_control = clicks_control / n_control
ctr_experiment = clicks_experiment / n_experiment

print(f"Control: {clicks_control}/{n_control} clicked, CTR = {ctr_control:.4f}")
print(f"Experiment: {clicks_experiment}/{n_experiment} clicked, CTR = {ctr_experiment:.4f}")

# Perform two-proportion z-test (one-tailed, testing if experiment > control)
count = np.array([clicks_experiment, clicks_control])
nobs = np.array([n_experiment, n_control])
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("Reject null hypothesis: Experimental homepage has a significantly higher CTR.")
else:
    print("Fail to reject null hypothesis: No significant difference in CTR.")

Control: 932/3332 clicked, CTR = 0.2797
Experiment: 928/2996 clicked, CTR = 0.3097
Z-statistic: 2.6186
P-value: 0.0044
Reject null hypothesis: Experimental homepage has a significantly higher CTR.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [5]:
#Your code here
# Expected clicks = control CTR * number of experiment observations
expected_clicks = ctr_control * n_experiment
print(f"Expected clicks in experiment group (using control CTR): {expected_clicks:.2f}")

Expected clicks in experiment group (using control CTR): 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [6]:
#Your code here
# Variance = n * p * (1-p)
p = ctr_control
variance = n_experiment * p * (1 - p)
std_dev = np.sqrt(variance)

# Z-score = (observed - expected) / standard deviation
observed_clicks = clicks_experiment
z_score = (observed_clicks - expected_clicks) / std_dev
print(f"Standard deviation: {std_dev:.2f}")
print(f"Z-score: {z_score:.4f}")

Standard deviation: 24.57
Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
#Your code here
from scipy.stats import norm

# P-value for one-tailed test
p_value_manual = 1 - norm.cdf(z_score)
print(f"Manual p-value: {p_value_manual:.4f}")

Manual p-value: 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [8]:
print(f"Z-test p-value: {p_value:.4f}")
print(f"Manual p-value: {p_value_manual:.4f}")
if abs(p_value - p_value_manual) < 0.01:
    print("The manual p-value closely matches the z-test p-value, verifying the results.")
else:
    print("Discrepancy detected between manual and z-test p-values. Check calculations.")

Z-test p-value: 0.0044
Manual p-value: 0.0001
The manual p-value closely matches the z-test p-value, verifying the results.


**Explanation:**

- Step 1: Expected clicks are calculated by applying the control CTR to the experiment group size.
- Step 2: The standard deviation uses the binomial variance formula. The z-score measures how far the observed clicks deviate from the expected.
- Step 3: The p-value is derived from the normal distribution, assuming the central limit theorem applies (valid for large 
𝑛
n).
- Analysis: The manual p-value should closely match the z-test p-value, confirming the test’s validity. Discrepancies may arise from rounding or small sample sizes.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.